# Unified ETL Pipeline

This notebook reproduces the steps from the Python ETL scripts in a single,
well-documented workflow. It generates the same output tables as the original
scripts for ONET, AEI and CAGED data.

## 1. Setup

In [ ]:

import os
import unicodedata
import pandas as pd
import numpy as np
import basedosdados as bd
from dotenv import load_dotenv

# OK
def strip_accents(text: str) -> str:
    """Return ``text`` without accent marks."""

    # 1) Decompose characters into base + combining marks (NFKD)
    #    e.g. "á" → "á"  (two code-points)
    decomposed = unicodedata.normalize("NFKD", text)
    # 2) Keep only the base characters (category != Mn = "Mark, Non-spacing")
    return "".join(ch for ch in decomposed if unicodedata.category(ch) != "Mn")


def load_onet_tasks_with_soc() -> pd.DataFrame:
    """Merge O*NET task statements with SOC major group titles.

    The CSV paths are fixed inside the function and point to the files in
    ``data/input/aei_data``.

    Returns
    -------
    pandas.DataFrame
        DataFrame with task statements and their corresponding SOC major group
        titles.
    """
    onet_df = pd.read_csv('data/input/aei_data/onet_task_statements.csv')
    onet_df['soc_major_group'] = onet_df['O*NET-SOC Code'].str[:2]

    soc_df = pd.read_csv('data/input/aei_data/SOC_Structure.csv')
    soc_df = soc_df.dropna(subset=['Major Group'])
    soc_df['soc_major_group'] = soc_df['Major Group'].str[:2]

    merged = onet_df.merge(
        soc_df[['soc_major_group', 'SOC or O*NET-SOC 2019 Title']],
        on='soc_major_group',
        how='left'
    )
    return merged


## 2. Loading

This section loads all datasets required for the ETL process.
The first code cell reads AEI task usage percentages, task classifications
and the SOC structure files. The second cell loads the CAGED dataset from
a cached parquet file (or queries it from BigQuery if the code is uncommented).

In [ ]:
# Percent of workers using each task
task_usage_pct = pd.read_csv('data/input/aei_data/task_usage_pct_v2.csv')
# Automation vs augmentation scores for each task
auto_aug_by_task = pd.read_csv(
    'data/input/aei_data/auto_aug_by_task.csv'
)
# Raw O*NET task statements
onet_tasks = pd.read_csv('data/input/aei_data/onet_task_statements.csv')
# SOC titles linked to each task
soc_structure_tasks = pd.read_csv('data/input/aei_data/SOC_Structure.csv')

# Full SOC hierarchy used for ONET lookup
soc_structure_full = pd.read_csv('data/input/SOC_Structure.csv')


In [ ]:
# Loading CAGED
# Load environment variables for BigQuery access
load_dotenv()
# Set a row limit when re-running the SQL query
LIMIT = False


# To re-run the query from the original data source, uncomment the lines below
# billing_id = os.getenv('BILLING_ID')
# with open('data/config/caged_bd_national.SQL') as f:
#     query = f.read()
#     if LIMIT:
#         query += f" LIMIT {LIMIT}"
# df = bd.read_sql(query=query, billing_project_id=billing_id)
# df.to_parquet('data/input/caged_national_UNTREATED.parquet')

# Cached CAGED query result
caged_raw = pd.read_parquet('data/input/caged_national_UNTREATED.parquet')

## 3. ONET ETL

In [3]:

# Extract the minor group prefix from each row
soc_structure_full['minor_group'] = soc_structure_full.apply(
    lambda row: row.dropna().iloc[0][:4], axis=1
)

# Groups by minor group, aggregating "SOC or O*NET-SOC 2019 Title" by concatenating the strings with '; '
# Aggregate SOC titles by minor group
minor_groups = (
    soc_structure_full.groupby('minor_group')
    .agg({'SOC or O*NET-SOC 2019 Title': lambda x: '; '.join(x)})
    .rename({'SOC or O*NET-SOC 2019 Title': 'title'}, axis='columns')
)
minor_groups.to_csv('data/output/onet_minor_groups.csv', index=True)


## 4. AEI ETL

Generate AEI occupation level metrics.

This script merges Anthropic Economic Index (AEI) task data with the
Standard Occupational Classification (SOC) structure in order to compute
automation and augmentation ratios by SOC minor group. The resulting
tables are written to ``data/output`` for further analysis.

### 4.1 Merge AEI tasks with SOC data
This step combines task statements with SOC titles and computes usage percentages per task.

In [ ]:
# Combined task statements with SOC titles
tasks_with_soc = load_onet_tasks_with_soc()
# Normalized task names for joining
tasks_with_soc['task_normalized'] = tasks_with_soc['Task'].str.lower().str.strip()

tasks_with_soc['num_occupations'] = (
    tasks_with_soc.groupby('task_normalized')['Title'].transform('nunique')
)

# Merge usage percentages with task metadata
task_usage = task_usage_pct.merge(
    tasks_with_soc, left_on='task_name', right_on='task_normalized', how='left'
)

# Scale usage by how many occupations perform the task
task_usage['pct_occ_scaled'] = 100 * (
    task_usage['pct'] / task_usage['num_occupations']
) / (
    task_usage['pct'] / task_usage['num_occupations']
).sum()

# Add automation vs augmentation labels
auto_aug_usage = task_usage.merge(
    auto_aug_by_task, on='task_name', how='left'
)
assert len(auto_aug_usage) == len(task_usage)


### 4.2 Aggregate metrics by occupation
Using the merged task data we compute weighted automation and augmentation scores for each SOC minor group.
Scores are calculated by summing the task-level contributions after weighting them by task usage percentages.

In [ ]:
# Map each task to its SOC minor group
minor_task_lookup = onet_tasks[['O*NET-SOC Code', 'Task']].copy()
# Minor group code from the SOC code
minor_task_lookup['soc_minor_group'] = minor_task_lookup['O*NET-SOC Code'].str[:4]
# Normalized task name
minor_task_lookup['task_name'] = minor_task_lookup['Task'].str.lower().str.strip()
# Map each task to its SOC minor group
# Keep unique task to minor group mappings
minor_task_lookup = minor_task_lookup[['soc_minor_group', 'task_name']].drop_duplicates()

# Combine usage and automation/augmentation info
task_metrics = task_usage_pct.merge(auto_aug_by_task, on='task_name', how='left')
# Replace missing scores with zero
task_metrics.fillna(0, inplace=True)
# Weighted augmentation score
task_metrics['aug'] = (task_metrics['learning'] + task_metrics['validation']) * task_metrics['pct']
# Weighted automation score
task_metrics['aut'] = (task_metrics['feedback_loop'] + task_metrics['directive'] + task_metrics['task_iteration']) * task_metrics['pct']
# Remove intermediate columns
task_metrics.drop(columns=['learning', 'validation', 'feedback_loop', 'directive', 'task_iteration'], inplace=True)

# Combine usage and automation/augmentation info
# Attach minor group info to each task
task_metrics = task_metrics.merge(minor_task_lookup, on='task_name', how='left')
# Sum metrics for each minor group
occupation_metrics = task_metrics.groupby('soc_minor_group')[['pct', 'aug', 'aut']].sum().reset_index()
# Ratio of augmentation to automation
occupation_metrics['aug_aut_ratio'] = occupation_metrics['aug'] / occupation_metrics['aut']
# Save aggregated metrics
occupation_metrics.to_parquet('data/output/occ_aut_aug_lvl.parquet')

### 4.3 Classify occupations
Based on the aggregated metrics we identify occupations with extreme values.
The following lists capture the occupations with the highest and lowest overall task usage,
as well as those most associated with automation or augmentation.

In [ ]:

# Classifying jobs
## 1. Highest usage jobs
# Occupations where AEI tasks are most prevalent
highest_pct_occupations = occupation_metrics.sort_values('pct', ascending=False).head(10)
highest_pct_occupations['class'] = 'Top 10 pct'

## 2. Lowest usage jobs
# Occupations where AEI tasks are least common
lowest_pct_occupations = occupation_metrics.sort_values('pct', ascending=True).head(10)
lowest_pct_occupations['class'] = 'Bottom 10 pct'

## 3. jobs with highest aut
# Occupations with the greatest automation usage
highest_aut_occupations = occupation_metrics.sort_values('aut', ascending=False).head(10)
highest_aut_occupations['class'] = 'Top 10 aut'

## 4. jobs with highest aug
# Occupations with the greatest augmentation usage
highest_aug_occupations = occupation_metrics.sort_values('aug', ascending=False).head(10)
highest_aug_occupations['class'] = 'Top 10 aug'

# Combine all rankings into a single table
classified_occupations = pd.concat([highest_pct_occupations, lowest_pct_occupations, highest_aut_occupations, highest_aug_occupations])
classified_occupations.to_parquet('data/output/occ_aut_aug_lvl_classified.parquet')

## 5. CAGED ETL

In [ ]:

# Clean CAGED job data
caged = caged_raw.copy()
# Remove rows missing occupation information
caged = caged[~caged['cbo_2002_descricao_subgrupo_principal'].isna()]

# Columns used for aggregation
cols_to_keep = [
    'ano',
    'mes',
    'cbo_2002_descricao_subgrupo_principal',
    'cbo_2002_descricao_grande_grupo'
]

# Aggregate monthly job flows
caged = (
    caged.groupby(cols_to_keep, dropna=False)['saldo_movimentacao']
    .sum()
    .reset_index()
    .apply(lambda col: col.str.strip() if col.dtype == 'object' else col)
)

# Rename columns for clarity
caged = caged.rename(
    columns={
        'saldo_movimentacao': 'net_jobs',
        'ano': 'year',
        'mes': 'month',
        'cbo_2002_descricao_subgrupo_principal': 'cbo_subgroup',
        'cbo_2002_descricao_grande_grupo': 'cbo_group'
    }
)

# Normalize occupation names
caged['cbo_subgroup'] = caged['cbo_subgroup'].apply(strip_accents)
caged['cbo_group'] = caged['cbo_group'].apply(strip_accents)

# Persist the cleaned CAGED dataset
caged.to_parquet('data/input/caged_national.parquet')
